# folium 勉強

foliumとは
* foliumはleaflet.jsというJavascriptで使用することのできるマップをPythonライブラリ化したもの
* Open street map に標準的には可視化

参考サイト
* ドキュメント　https://qiita.com/Kumanuron-1910/items/12ce7aa02922927de2f4
* https://qiita.com/Kumanuron-1910/items/12ce7aa02922927de2f4

In [1]:
import folium
import pandas as pd
import geopandas as gpd

##  基本操作
* 指定したマーカーを定義した地図上に可視化が可能
* 定義した地図に可視化したいポイントを追加していくイメージ

In [2]:
# 地図生成
GAT = [35.70044027033906, 139.7505994642519]
folium_map = folium.Map(location=GAT,zoom_start=15)

# マーカープロット（ポップアップ設定，色変更，アイコン変更）
folium.Marker(
    location=GAT,
    popup='GAT',
    icon=folium.Icon(color='red')
).add_to(folium_map)

# サークルマーカープロット(外径色は赤，内径色は青)
folium.CircleMarker(
    location=GAT,
    radius=40,
    color='#ff0000',
    fill_color='#0000ff'
).add_to(folium_map)

# 地図表示
folium_map

In [3]:
# データ格納（名前，緯度経度）
df = pd.DataFrame({
    'station': ['新宿駅', '渋谷駅', '上野駅', '東京駅', '高輪ゲートウェイ駅'],
    'latitude': [35.690921, 35.658034, 35.713768, 35.681236, 35.6355406],
    'longtude': [139.700258, 139.701636, 139.777254, 139.767125, 139.7407245],
})

# 地図生成（新宿駅中心）
folium_map = folium.Map(location=[35.690921, 139.700258], zoom_start=12)

# マーカープロット
for i, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longtude']],
        popup=row['station'],
        icon=folium.Icon(color='red')
    ).add_to(folium_map)

# 地図表示
folium_map

## geojsonファイル読み込み
* geojsonとはテキストでPolyginなどの情報を保存
* shpファイルは複数のファイルが必要となるためよりシンプルに位置情報を管理することが可能
* https://www.gis-py.com/entry/gis-file-format

* 読み込みファイル：https://github.com/txslgr/geojson

In [4]:
geo_data = gpd.read_file("data/city_geojson.geojson", encoding="utf8")
geo_data.head(3)

,cartodb_id,cityname_k,prefname_k,_5keta_code1,city_code,cityname_e,prefname_e,geometry
0,688,利島村,東京都,133621,13362,Toshima Mura,Tokyo To,"MULTIPOLYGON (((139.27530 34.51114, 139.26849 ..."
1,727,愛川町,神奈川県,144011,14401,Aikawa Machi,Kanagawa Ken,"MULTIPOLYGON (((139.35440 35.52753, 139.35170 ..."
2,728,清川村,神奈川県,144029,14402,Kiyokawa Mura,Kanagawa Ken,"MULTIPOLYGON (((139.24809 35.54246, 139.24680 ..."


In [5]:
# 指定の市区町村をプロット
geo_data_tokyo = geo_data[geo_data["prefname_k"]=="神奈川県"]
folium_map = folium.Map(location=GAT, zoom_start=9)
folium.GeoJson(data=geo_data_tokyo["geometry"]).add_to(folium_map) 
folium_map

## タイルの切り替え

In [6]:
japan_map = folium.Map(location=[35.6679,139.4606],zoom_start=8)

folium.raster_layers.TileLayer(
    tiles = 'https://cyberjapandata.gsi.go.jp/xyz/std/{z}/{x}/{y}.png',
    fmt = 'image/png',
    attr = '&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">国土地理院</a>',
    name = '国土地理院地図'
).add_to(japan_map)

folium.raster_layers.TileLayer(
    tiles = 'https://gbank.gsj.jp/seamless/v2/api/1.2.1/tiles/{z}/{y}/{x}.png', 
    fmt = 'image/png',
    attr = '&copy; <a href="https://gbank.gsj.jp/seamless/v2/api/1.2.1/">日本シームレス地質図V2 ( GSJ, AIST )</a>',
    name = '日本シームレス地質図V2'
).add_to(japan_map)

folium.LayerControl().add_to(japan_map)

japan_map